<a href="https://colab.research.google.com/github/kzemla/dw_matrix_road_sigh/blob/master/day5_road_sign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install hyperopt

In [46]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sigh'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sigh


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])



  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      vaidation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
 
  )

  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score ) :


  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
   MaxPool2D(), 
   Dropout(0.3),

   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
   MaxPool2D(), 
   Dropout(0.3),

   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
   MaxPool2D(), 
   Dropout(0.3),


   Flatten(),

   Dense(1024, activation='relu'),
   Dropout(0.3),
   
   Dense(1024, activation='relu'),
   Dropout(0.3),


   Dense(num_classes, activation='softmax'),
   ])


In [52]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 3s 10ms/step - loss: 2.4553 - accuracy: 0.3250
Epoch 2/5
272/272 [==============================] - 3s 9ms/step - loss: 0.7442 - accuracy: 0.7674
Epoch 3/5
272/272 [==============================] - 3s 9ms/step - loss: 0.3624 - accuracy: 0.8870
Epoch 4/5
272/272 [==============================] - 3s 9ms/step - loss: 0.2404 - accuracy: 0.9281
Epoch 5/5
272/272 [==============================] - 3s 9ms/step - loss: 0.1913 - accuracy: 0.9426


0.9575963718820861

In [53]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.2386 - accuracy: 0.9576


[0.2385822832584381, 0.9575963616371155]

In [0]:
def func_obj(params):

  model = get_model(params)
  model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose=0
      
 
  )
  score =  model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model }

In [64]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
    


}

best= fmin(
    func_obj, 
    space,
    tpe.suggest, 
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.33209438978466926, 'dropout_cnn_block_three': 0.4816732506050308, 'dropout_cnn_block_two': 0.3395478372673721, 'dropout_dense_block_one': 0.39714334083499286, 'dropout_dense_block_two': 0.6694092004049654}
accuracy=0.9700680375099182
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3302381081879255, 'dropout_cnn_block_three': 0.43585465736476803, 'dropout_cnn_block_two': 0.45584281015959466, 'dropout_dense_block_one': 0.5480042015001774, 'dropout_dense_block_two': 0.35597297620959206}
accuracy=0.9609977602958679
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.35079549137604815, 'dropout_cnn_block_three': 0.48996690513352625, 'dropout_cnn_block_two': 0.376783073205911, 'dropout_dense_block_one': 0.6019804332422757, 'dropout_dense_block_two': 0.6564008411580757}
accuracy=0.9503401517868042
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4559708598823353, 'dropout_cnn_block_three': 0.4910506402235433, 'dropout_cnn_block_two': 0.3986764353

In [0]:
def get_model(params):
  return Sequential([
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
   MaxPool2D(), 
   Dropout(params['dropout_cnn_block_one']),

   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
   MaxPool2D(), 
   Dropout(params['dropout_cnn_block_two']),


   Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=128, kernel_size=(3,3), activation='relu'), 
   MaxPool2D(), 
   Dropout(params['dropout_cnn_block_three']),


   Flatten(),

   Dense(1024, activation='relu'),
   Dropout(params['dropout_dense_block_one']),

   
   Dense(1024, activation='relu'),
   Dropout(params['dropout_cnn_block_two']),



   Dense(num_classes, activation='softmax'),
   ])


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
   Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
   MaxPool2D(), 
   Dropout(0.3),

   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
   Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
   MaxPool2D(), 
   Dropout(0.3),

   Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'), 
   Conv2D(filters=128, kernel_size=(3,3), activation='relu'), 
   MaxPool2D(), 
   Dropout(0.3),


   Flatten(),

   Dense(1024, activation='relu'),
   Dropout(0.3),
   
   Dense(1024, activation='relu'),
   Dropout(0.3),


   Dense(num_classes, activation='softmax'),
   ])


In [66]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 25ms/step - loss: 2.2910 - accuracy: 0.3423
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.5339 - accuracy: 0.8292
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2307 - accuracy: 0.9299
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1625 - accuracy: 0.9537
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1209 - accuracy: 0.9659


0.9705215419501134